In [2]:
import numpy as np
import pandas as pd

In [3]:
temp_df = pd.read_csv('IMDB Dataset.csv')

In [4]:
df = temp_df.iloc[:10000]

In [5]:
df.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [6]:
df.drop_duplicates(inplace=True)

C:\ProgramData\Miniconda3\lib\site-packages\pandas\util\_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)


In [7]:
import re
def remove_tags(raw_text):
    cleaned_text = re.sub(re.compile('<.*?>'), '', raw_text)
    return cleaned_text

In [8]:
df['review'] = df['review'].apply(remove_tags)

C:\Users\hp\AppData\Local\Temp/ipykernel_15884/2336150696.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['review'] = df['review'].apply(remove_tags)


In [9]:
df['review'] = df['review'].apply(lambda x:x.lower())

C:\Users\hp\AppData\Local\Temp/ipykernel_15884/740760900.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['review'] = df['review'].apply(lambda x:x.lower())


In [10]:
from nltk.corpus import stopwords

sw_list = stopwords.words('english')

df['review'] = df['review'].apply(lambda x: [item for item in x.split() if item not in sw_list]).apply(lambda x:" ".join(x))

C:\Users\hp\AppData\Local\Temp/ipykernel_15884/2826946130.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['review'] = df['review'].apply(lambda x: [item for item in x.split() if item not in sw_list]).apply(lambda x:" ".join(x))


In [11]:
df['review']

0       one reviewers mentioned watching 1 oz episode ...
1       wonderful little production. filming technique...
2       thought wonderful way spend time hot summer we...
3       basically there's family little boy (jake) thi...
4       petter mattei's "love time money" visually stu...
                              ...                        
9995    fun, entertaining movie wwii german spy (julie...
9996    give break. anyone say "good hockey movie"? kn...
9997    movie bad movie. watching endless series bad h...
9998    movie probably made entertain middle school, e...
9999    smashing film film-making. shows intense stran...
Name: review, Length: 9983, dtype: object

In [12]:
import gensim

In [13]:
from nltk import sent_tokenize
from gensim.utils import simple_preprocess

In [14]:
story = []
for doc in df['review']:
    raw_sent = sent_tokenize(doc)
    for sent in raw_sent:
        story.append(simple_preprocess(sent))
    

IndexError: list index out of range

In [16]:
model = gensim.models.Word2Vec(
    window=10,
    min_count=2
)

In [17]:
model.build_vocab(story)

In [18]:
model.train(story, total_examples=model.corpus_count, epochs=model.epochs)

(747604, 827360)

In [19]:
len(model.wv.index_to_key)

11105

In [20]:
def document_vector(doc):
    # remove out-of-vocabulary words
    doc = [word for word in doc.split() if word in model.wv.index_to_key]
    return np.mean(model.wv[doc], axis=0)

In [21]:
document_vector(df['review'].values[0])

array([-0.22272393,  0.35590926,  0.19250162,  0.11025099,  0.0390838 ,
       -0.71490383,  0.21090512,  0.8845256 , -0.33278623, -0.21969844,
       -0.3640085 , -0.7135408 , -0.07217435,  0.2840993 ,  0.17640589,
       -0.37249562,  0.1660485 , -0.32463035,  0.01034309, -0.7164344 ,
        0.16635491,  0.0173232 ,  0.37833685, -0.26433474,  0.02447818,
        0.00474235, -0.3029959 , -0.19827817, -0.44812295, -0.01107626,
        0.31617573,  0.06169416,  0.23917219, -0.24304825, -0.12341073,
        0.3453804 , -0.02858324, -0.30096343, -0.18309975, -0.71567374,
        0.1963034 , -0.3799625 , -0.15732008, -0.04418311,  0.35022283,
       -0.2892099 , -0.5210918 , -0.02836978,  0.09972842,  0.2709962 ,
        0.30632997, -0.3408223 , -0.11737426,  0.00837049, -0.23064588,
        0.19519773,  0.27508116,  0.06941223, -0.405903  ,  0.08149083,
        0.03844983,  0.08766174, -0.08792187,  0.22266993, -0.34423226,
        0.33812684,  0.13312486,  0.24101597, -0.34414864,  0.49

In [22]:
from tqdm import tqdm

In [23]:
X = []
for doc in tqdm(df['review'].values):
    X.append(document_vector(doc))

100%|██████████| 9983/9983 [01:15<00:00, 131.53it/s]


In [24]:
X = np.array(X)

In [25]:
X[0]

array([-0.22272393,  0.35590926,  0.19250162,  0.11025099,  0.0390838 ,
       -0.71490383,  0.21090512,  0.8845256 , -0.33278623, -0.21969844,
       -0.3640085 , -0.7135408 , -0.07217435,  0.2840993 ,  0.17640589,
       -0.37249562,  0.1660485 , -0.32463035,  0.01034309, -0.7164344 ,
        0.16635491,  0.0173232 ,  0.37833685, -0.26433474,  0.02447818,
        0.00474235, -0.3029959 , -0.19827817, -0.44812295, -0.01107626,
        0.31617573,  0.06169416,  0.23917219, -0.24304825, -0.12341073,
        0.3453804 , -0.02858324, -0.30096343, -0.18309975, -0.71567374,
        0.1963034 , -0.3799625 , -0.15732008, -0.04418311,  0.35022283,
       -0.2892099 , -0.5210918 , -0.02836978,  0.09972842,  0.2709962 ,
        0.30632997, -0.3408223 , -0.11737426,  0.00837049, -0.23064588,
        0.19519773,  0.27508116,  0.06941223, -0.405903  ,  0.08149083,
        0.03844983,  0.08766174, -0.08792187,  0.22266993, -0.34423226,
        0.33812684,  0.13312486,  0.24101597, -0.34414864,  0.49

In [26]:
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()

y = encoder.fit_transform(df['sentiment'])

In [27]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=1)

In [28]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

In [29]:
rf = RandomForestClassifier()
rf.fit(X_train,y_train)
y_pred = rf.predict(X_test)
accuracy_score(y_test,y_pred)

0.6124186279419128